In [26]:
import sys
import pandas as pd
sys.path.append("C:/Users/ping/MyDrive/py_files/python/py379/")
from myUtils import pickle_load, pickle_dump
pd.set_option('max_colwidth', 12, 'display.max_columns', 18, 'display.width', 1200, 'display.max_rows', 100)
INPUT_DIR = 'C:/Users/ping/OneDrive/Documents/jenn_bb_sales'
path_pickle_dump = f'{INPUT_DIR}/'

In [27]:
df = pickle_load(path_pickle_dump, 'df_item_sale_n_weather_cleaned')
print(f'df.shape: {df.shape}')
print(f'df.columns: {df.columns}')

df.shape: (18364, 23)
df.columns: Index(['Date', 'Time', 'Category', 'Item', 'Qty', 'Gross Sales', 'Discounts', 'Net Sales', 'Tax', 'Transaction ID', 'Event Type', 'Dining Option', 'Customer Name', 'NAME', 'AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN', 'WT01', 'WT02', 'WT08', 'DOW'], dtype='object')


In [28]:
cols_drop = ['NAME', 'AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN', 'WT01', 'WT02', 'WT08', 'DOW']
df.drop(cols_drop, axis=1, inplace=True)
print(f'df.columns: {df.columns}')
print(f'df.shape: {df.shape}')

df.columns: Index(['Date', 'Time', 'Category', 'Item', 'Qty', 'Gross Sales', 'Discounts', 'Net Sales', 'Tax', 'Transaction ID', 'Event Type', 'Dining Option', 'Customer Name'], dtype='object')
df.shape: (18364, 13)


In [29]:
df

,Date,Time,Category,Item,Qty,Gross Sales,Discounts,Net Sales,Tax,Transaction ID,Event Type,Dining Option,Customer Name
0,2021-07-08,17:31:07,Food,Chocolat...,1.0,6.0,0.0,6.0,0.00,5Qzw7N4E...,Payment,To Go,Jennifer...
1,2021-07-08,17:31:29,Food,Jarred P...,1.0,6.0,0.0,6.0,0.00,Z4Lhlkfm...,Payment,For Here,Jennifer...
2,2021-07-08,17:31:47,Beverage,Sparkling,1.0,3.0,0.0,3.0,0.28,Zxzxigke...,Payment,For Here,Jennifer...
3,2021-07-08,17:32:35,Beverage,Sparkling,-1.0,3.0,0.0,3.0,0.28,Hzcnf0W9...,Refund,For Here,NaN
4,2021-07-08,17:32:54,Food,Jarred P...,-1.0,6.0,0.0,6.0,0.00,Zodxuuaa...,Refund,For Here,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18359,2022-09-04,11:19:02,Food,Cinnamon...,1.0,3.0,0.0,3.0,0.00,Lmk3E9Gx...,Payment,For Here,Aubrie D...
18360,2022-09-04,11:20:38,Food,Oatmeal ...,1.0,4.0,0.0,4.0,0.00,Hokel420...,Payment,For Here,Jean Sequen
18361,2022-09-04,11:21:25,Food,Cinnamon...,1.0,3.0,0.0,3.0,0.00,Huikykv6...,Payment,For Here,NaN
18362,2022-09-04,11:32:24,Food,Cinnamon...,1.0,3.0,0.0,3.0,0.00,Zdb6Eyob...,Payment,For Here,Yesenia ...


In [30]:
df = df[df['Event Type'] != 'Refund']
print(f'df.shape: {df.shape}')

df.shape: (18337, 13)


In [31]:
df = df[df['Qty'] >= 0]
print(f'df.shape: {df.shape}')

df.shape: (18337, 13)


In [ ]:
df

In [32]:
df.columns

Index(['Date', 'Time', 'Category', 'Item', 'Qty', 'Gross Sales', 'Discounts', 'Net Sales', 'Tax', 'Transaction ID', 'Event Type', 'Dining Option', 'Customer Name'], dtype='object')

In [34]:
_cols = ['Transaction ID', 'Date', 'Time', 'Category', 'Item', 'Qty', 'Gross Sales']
df1 = df[_cols]
pd.set_option('max_colwidth', 30) 
df1

,Transaction ID,Date,Time,Category,Item,Qty,Gross Sales
0,5Qzw7N4Epco9Ezmhvubnjawev,2021-07-08,17:31:07,Food,Chocolate Croissant,1.0,6.0
1,Z4Lhlkfmd1Wueqp2Etutjvvev,2021-07-08,17:31:29,Food,Jarred Pastry,1.0,6.0
2,Zxzxigkeb6Nw6Ytwrgrdzweev,2021-07-08,17:31:47,Beverage,Sparkling,1.0,3.0
6,Jwogqfj8U01Abzqlchlrplsev,2021-07-09,09:14:03,Food,Croissant,2.0,10.0
7,Jwogqfj8U01Abzqlchlrplsev,2021-07-09,09:14:03,Food,Chocolate Croissant,2.0,12.0
...,...,...,...,...,...,...,...
18359,Lmk3E9Gxitmoh4U6C8Qec55Ev,2022-09-04,11:19:02,Food,Cinnamon Knot,1.0,3.0
18360,Hokel4207Mxfkjwl16Mu6N6Ev,2022-09-04,11:20:38,Food,Oatmeal Cookie,1.0,4.0
18361,Huikykv6Itvipng8Cmty6F5Ev,2022-09-04,11:21:25,Food,Cinnamon Knot,1.0,3.0
18362,Zdb6Eyob4Zx1Lu8Jgxfstf6Ev,2022-09-04,11:32:24,Food,Cinnamon Knot,1.0,3.0


In [40]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
sales_by_TransID = df1.groupby(['Transaction ID', 'Date', 'Time', 'Category', 'Item']).agg(['sum'])
sales_by_TransID = sales_by_TransID.sort_values(['Date', 'Time'])
sales_by_TransID

Qty Gross Sales
                                                                            sum         sum
Transaction ID            Date       Time     Category Item                                
5Qzw7N4Epco9Ezmhvubnjawev 2021-07-08 17:31:07 Food     Chocolate Croissant  1.0         6.0
Z4Lhlkfmd1Wueqp2Etutjvvev 2021-07-08 17:31:29 Food     Jarred Pastry        1.0         6.0
Zxzxigkeb6Nw6Ytwrgrdzweev 2021-07-08 17:31:47 Beverage Sparkling            1.0         3.0
Jwogqfj8U01Abzqlchlrplsev 2021-07-09 09:14:03 Food     Blueberry Muffin     1.0         4.0
                                                       Brownie              1.0         5.0
...                                                                         ...         ...
Lmk3E9Gxitmoh4U6C8Qec55Ev 2022-09-04 11:19:02 Food     Danish               1.0         7.0
Hokel4207Mxfkjwl16Mu6N6Ev 2022-09-04 11:20:38 Food     Oatmeal Cookie       1.0         4.0
Huikykv6Itvipng8Cmty6F5Ev 2022-09-04 11:21:25 Food     Cinnamon Knot        1.0         3.0
Zdb6Eyob4Zx1Lu8Jgxfstf6Ev 2022-09-04 11:32:24 Food     Cinnamon Knot        1.0         3.0
J71Bpp0Ogcqppzk3Tdfczufev 2022-09-04 11:39:13 Food     Danish               2.0        14.0

[18197 rows x 2 columns]

In [41]:
sales_by_TransID.tail(30)

Qty Gross Sales
                                                                                sum         sum
Transaction ID            Date       Time     Category Item                                    
Xqdy4Jli2Mmyix3Gfe3Nqysev 2022-09-04 10:10:17 Food     Croissant                1.0         6.0
                                                       Danish                   1.0         7.0
Bh4Kolmuuffdwbhski1Zthoev 2022-09-04 10:11:45 Food     Cinnamon Knot            1.0         3.0
                                                       Croissant                1.0         6.0
                                                       Danish                   1.0         7.0
                                                       Oatmeal Cookie           1.0         4.0
Zglmcvvpczghu6Zk8Tjkow4Ev 2022-09-04 10:12:38 Food     Chocolate Croissant      1.0         7.0
Jyj6Rirq9Kfhdx0Humruz19Ev 2022-09-04 10:22:25 Food     Danish                   1.0         7.0
Hrndqiumgddmgryaxzdrfqmev 2022-09-04 10:25:43 Food     Chocolate Chip Cookie    1.0         4.0
                                                       Danish                   1.0         7.0
                                                       Double Chocolate Cookie  1.0         4.0
Rymq0Veinmvg5G45Wuh1F8Xev 2022-09-04 10:27:16 Food     Chocolate Croissant      1.0         7.0
                                                       Croissant                1.0         6.0
1K7Epnpnb9Ixszckhrkshajev 2022-09-04 10:28:52 Food     Danish                   2.0        14.0
                                                       Furikake Crx             1.0         7.0
J3Lsdefdmkr598Uoggcfgxkev 2022-09-04 10:33:39 Food     Chocolate Croissant      1.0         7.0
H76Ilpko6Pnq520Qho0Ogcpev 2022-09-04 10:40:45 Food     Chocolate Croissant      1.0         7.0
                                                       Cinnamon Knot            1.0         3.0
                                                       Furikake Crx             1.0         7.0
Jscitrte3Jcrybtzkum7Umyev 2022-09-04 10:48:54 Food     Cinnamon Knot            4.0        12.0
                                                       Danish                   4.0        28.0
Vxqr2Ucndp2X7Nzqtc28Byhev 2022-09-04 10:51:38 Food     Cinnamon Knot            5.0        15.0
3Fxjplkkb9Xulcdofnk0Umfev 2022-09-04 10:59:49 Food     Danish                   2.0        14.0
                                                       Furikake Crx             2.0        14.0
Lmk3E9Gxitmoh4U6C8Qec55Ev 2022-09-04 11:19:02 Food     Cinnamon Knot            1.0         3.0
                                                       Danish                   1.0         7.0
Hokel4207Mxfkjwl16Mu6N6Ev 2022-09-04 11:20:38 Food     Oatmeal Cookie           1.0         4.0
Huikykv6Itvipng8Cmty6F5Ev 2022-09-04 11:21:25 Food     Cinnamon Knot            1.0         3.0
Zdb6Eyob4Zx1Lu8Jgxfstf6Ev 2022-09-04 11:32:24 Food     Cinnamon Knot            1.0         3.0
J71Bpp0Ogcqppzk3Tdfczufev 2022-09-04 11:39:13 Food     Danish                   2.0        14.0

In [ ]:
# https://datagy.io/pandas-groupby/
for column in df.columns[2::]: # skip columns: Date, Time
  s = df[column]
  grouped = s.groupby(s)
  print(f'{column}')
  print('='*20)
  _dict = {}
  for key in grouped.groups.keys():
    _count = grouped.get_group(key).count()
    _dict[key] = _count
  
  # sort items and counts in the column in reverse order 
  _dict = dict(sorted(_dict.items(), reverse = True, key=lambda item: item[1]))
  for k, v in _dict.items():
      print(f'{k:<45}{v:10,.0f}')
  print('='*20, '\n')

In [ ]:
# how many rows have one or more valid inputs in columns: WT01, WT02, WT08'
_df = df[['WT01', 'WT02', 'WT08']]
_idx = _df.index[_df.notnull().any(axis=1)]
_df_notnull = _df.iloc[_idx]
print(f'{_df_notnull.shape[0]} rows have one or more valid inputs in columns: WT01, WT02, WT08')
print(f'_df_notnull.shape: {_df_notnull.shape}')